In [7]:
import pandas as pd

In [8]:
# Cargar el archivo CSV
df = pd.read_csv("../Gasolina/EDS.csv")

# Renombrar columnas
df.columns = ['AÑO', 'MES', 'DEPARTAMENTO', 'MUNICIPIO', 'PRODUCTO', 'VOLUMEN_DESPACHADO', 'PRECIO_PROMEDIO']

# Limpiar comas y convertir el precio a número
df['PRECIO_PROMEDIO'] = df['PRECIO_PROMEDIO'].astype(str).str.replace(',', '').astype(float)


In [9]:
# Diccionario para convertir el mes en texto a número
meses = {
    'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4,
    'MAYO': 5, 'JUNIO': 6, 'JULIO': 7, 'AGOSTO': 8,
    'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
}

df['MES_NUM'] = df['MES'].str.upper().map(meses)


In [10]:
# Calcular promedio del precio agrupando por año, mes y producto
promedios = (
    df.groupby(['AÑO', 'MES_NUM', 'PRODUCTO'], as_index=False)['PRECIO_PROMEDIO']
      .mean()
)


In [12]:
# Pivotar para tener una columna por tipo de producto
tabla_final = promedios.pivot_table(
    index=['AÑO', 'MES_NUM'],
    columns='PRODUCTO',
    values='PRECIO_PROMEDIO'
).reset_index()

# Asegurar que las columnas no sean MultiIndex
tabla_final.columns.name = None

# Renombrar automáticamente las columnas con nombres limpios
tabla_final = tabla_final.rename(columns=lambda x: (
    "PRECIO_PROMEDIO_" + x.split()[-1].upper()
    if x not in ['AÑO', 'MES_NUM'] else x
))

# Mostrar nombres reales de las columnas para confirmar
print("🧩 Columnas resultantes:", tabla_final.columns.tolist())

# Vista previa
tabla_final.head()


🧩 Columnas resultantes: ['AÑO', 'MES_NUM', 'PRECIO_PROMEDIO_MEZCLA', 'PRECIO_PROMEDIO_MARINO', 'PRECIO_PROMEDIO_CORRIENTE', 'PRECIO_PROMEDIO_EXTRA']


,AÑO,MES_NUM,PRECIO_PROMEDIO_MEZCLA,PRECIO_PROMEDIO_MARINO,PRECIO_PROMEDIO_CORRIENTE,PRECIO_PROMEDIO_EXTRA
0,2021,1,8456.102537,NaN,8481.725207,11470.058091
1,2021,2,8529.885835,NaN,8583.135052,11542.452675
2,2021,3,8644.089757,NaN,8759.719588,11741.618257
3,2021,4,8687.367476,NaN,8790.843847,12110.850427
4,2021,5,8744.867663,NaN,8890.243271,12177.167442


In [13]:
# Crear una copia para evitar modificar accidentalmente el original
tabla_limpia = tabla_final.copy()

# Eliminar la columna de producto marino si existe
if 'PRECIO_PROMEDIO_MARINO' in tabla_limpia.columns:
    tabla_limpia.drop(columns=['PRECIO_PROMEDIO_MARINO'], inplace=True)

# Renombrar la columna mezcla a biodiésel si existe
tabla_limpia.rename(
    columns={'PRECIO_PROMEDIO_MEZCLA': 'PRECIO_PROMEDIO_BIODIESEL'},
    inplace=True
)

# Mostrar las columnas finales
print("✅ Columnas finales:", tabla_limpia.columns.tolist())

# Vista previa de la tabla limpia
tabla_limpia.head()


✅ Columnas finales: ['AÑO', 'MES_NUM', 'PRECIO_PROMEDIO_BIODIESEL', 'PRECIO_PROMEDIO_CORRIENTE', 'PRECIO_PROMEDIO_EXTRA']


,AÑO,MES_NUM,PRECIO_PROMEDIO_BIODIESEL,PRECIO_PROMEDIO_CORRIENTE,PRECIO_PROMEDIO_EXTRA
0,2021,1,8456.102537,8481.725207,11470.058091
1,2021,2,8529.885835,8583.135052,11542.452675
2,2021,3,8644.089757,8759.719588,11741.618257
3,2021,4,8687.367476,8790.843847,12110.850427
4,2021,5,8744.867663,8890.243271,12177.167442


In [14]:
# Guardar la tabla final en un nuevo CSV
tabla_limpia.to_csv("../Gasolina/promedios_mensuales_productos.csv", index=False)

print("✅ Archivo guardado en Gasolina/promedios_mensuales_productos.csv")


✅ Archivo guardado en Gasolina/promedios_mensuales_productos.csv
